In [11]:
"""Mutual information estimator from
- `"Understanding autoencoders with..." <https://arxiv.org/pdf/1804.00057.pdf>`_ paper
- `"Understanding Convolutional num_sampleseural num_samplesetworks with..."
    <https://arxiv.org/pdf/1804.06537.pdf>`_ paper.

Based on the implementation in the above two papers:
- https://drive.google.com/drive/folders/1e5sIywZfmWp4Dn0WEesb6fqQRM0DIGxZ
- https://drive.google.com/drive/folders/1SlxzEOX8RbnLwCgRyqGwMOL7vuT90Gje
"""
import numpy as np


def guassian_matrix(X, sigma):
    '''Apply a guassian kernel on a realization X.

    Args:
        X: a realization of a layer activation distribution.
        sigma: parameter of the guassian kernel.

    Returns:
        Gram matrix K.
    '''
    G = np.matmul(X, X.T)
    K = np.exp(-(1/(2*sigma**2)) * (np.diag(G)[:, np.newaxis]
                                    + np.diag(G)[np.newaxis, :] - 2*G))

    return K

def silvermanmi_univariate(inputdata, layerdata, alpha):
    '''Calculate mutual information.

    Args:
        inputdata: Activations of input with dimension
            (args.num_samples, input dimension).
        layerdata: Activations of a hidden layer with dimension
            (args.num_samples, layer dimension).
        alpha: alpha in the definition of matrix based renyi entropy.

    Returns:
        mutual information
    '''
    num_samples = inputdata.shape[0]
    sigma1 = 6 * np.power(num_samples, -1/(4 + inputdata.shape[1]))
    sigma2 = 6 * np.power(num_samples, -1/(4 + layerdata.shape[1]))
    print(sigma1, sigma2)
    # Estimate entropy
    K_x = np.real(guassian_matrix(inputdata, sigma1)) / num_samples
    L_x, _ = np.linalg.eig(K_x)
    lambda_x = np.absolute(L_x)
    H_x = (1 / (1-alpha)) * np.log((np.sum(np.power(lambda_x, alpha))))

    K_t = np.real(guassian_matrix(layerdata, sigma2)) / num_samples
    L_t, _ = np.linalg.eig(K_t)
    lambda_t = np.absolute(L_t)
    H_t = (1 / (1-alpha)) * np.log((np.sum(np.power(lambda_t, alpha))))

    # Estimate joint entropy
    K_xt = np.multiply(K_x, K_t) * num_samples
    L_xt, _ = np.linalg.eig(K_xt)
    lambda_xt = np.absolute(L_xt)
    H_xt = (1 / (1-alpha)) * np.log(np.sum(np.power(lambda_xt, alpha)))

    # Estimate mutual information
    mutual_information = H_x + H_t - H_xt

    return mutual_information


## test the function silvermanmi_univariate()

In [12]:
alpha = 1.01
inputdata= np.array([[1],[2],[3]])
layerdata = np.array([[1],[2],[3]])
inputdata.shape

(3, 1)

In [13]:
mi = silvermanmi_univariate(inputdata, layerdata, alpha)
print("mi: ",mi)

4.816449370561385 4.816449370561385
mi:  0.0416007116536366


## calculate MI in Ising model

In [14]:
fnames = ["slices_L1000_D2_T1.5000_pBC",
          "slices_L1000_D2_T2.3000_pBC",
          "slices_L1000_D2_T2.2692_pBC",
          "slices_L1000_D2_T3.5000_pBC"]

In [ ]:
MIs = [[],[],[],[]]

alpha = 1.01
for n in range(4):
    data = np.loadtxt(fnames[n], delimiter=None)[:,:100]
    # (MCTOT, Linear_size)
    print(data.shape)
    inputdata = data[:,0].reshape(data.shape[0],1)
    for i in range(1,data.shape[1]):
        print(i)
        layerdata = data[:,i].reshape(data.shape[0],1)
        mi = silvermanmi_univariate(inputdata, layerdata, alpha)
        MIs[n].append(mi)

In [59]:
import matplotlib.pyplot as plt

In [ ]:
markers = ['r-','g-','y-','b-']
for n in range(4):
    plt.plot(list(range(1,100)),MIs[n],markers[n])